In [1]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64

In [38]:
# Load dataset
df = pd.read_csv('data/news.csv')

In [40]:
# Menampilkan 5 data pertama yang memiliki label 1
df[df['real_news'] == 1].head()

,tweet,real_news
0,zamalek continues its preparations for the el ...,1
1,"sheikh salem al-amry, vice president of the ar...",1
2,the arab contractors tested 3 foreigners under...,1
3,an urgent decision for caf today regarding the...,1
4,al-masry refuses to postpone the al-ahly match...,1


In [41]:
# Menampilkan 5 data pertama yang memiliki label 0
df[df['real_news'] == 0].head()

,tweet,real_news
1000,he played almost over 8 matches against chelse...,0
1001,plan b: the ranger is our boy..,0
1002,do you think that we are talking seriously and...,0
1003,grandpa ferrera tells you that zamalek is stre...,0
1004,a statement by gamal al-ghandour with captain ...,0


In [42]:
tweet = df['tweet'][1500]

In [43]:
def prepare_json(tweet):
    """Mempersiapkan data dalam format JSON untuk dikirim ke model TensorFlow Serving."""
    feature_spec = {
        'tweet': tf.train.Feature(
            bytes_list=tf.train.BytesList(value=[bytes(tweet, 'utf-8')])
        )
    }
    
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    
    result = [{
        'examples': {
            'b64': base64.b64encode(example).decode()
        }
    }]
    
    return json.dumps({
        'signature_name': 'serving_default',
        'instances': result
    })

In [44]:
def predict(text):
    """Mengirim data ke model yang berjalan di TensorFlow Serving dan mendapatkan prediksi."""
    response = requests.post(
        'http://localhost:8080/v1/models/Fake_News_football_model:predict',
        data=prepare_json(text)
    )
    
    prediction = response.json()['predictions'][0][0]
    return prediction

# Prediksi
prediction = predict(tweet)

# Output hasil prediksi
print(f'{tweet}\n')
if prediction > 0.6:
    print(f'This is Real News ({prediction:.4f})')
else:
    print(f'This is Fake News ({prediction:.4f})')

# al-nasr_ al-qadisiyah this is al-nasr club, nor a foreign club from the goalkeeper to the attack. i do not know any of them, they are all foreigners. i only knew omar hawsawi.

This is Fake News (0.0024)
